# Multimodal RAG System Demo

This notebook demonstrates how to use the multimodal RAG system to process and query documents, images, and audio files.

## 1. Setup and Configuration

First, let's import the necessary modules and configure the system.

In [ ]:
import sys
from pathlib import Path
import asyncio

# Add the src directory to the path
sys.path.append(str(Path('../src')))

from multimodal_rag import MultimodalRAG, RAGConfig
from loguru import logger

# Configure logging
logger.add("../logs/demo.log", rotation="1 MB")

print("✅ Imports successful!")

In [ ]:
# Load configuration
config_path = "../config/config.yaml"
config = RAGConfig.from_yaml(config_path)

# Display configuration
print("📋 Configuration loaded:")
print(f"  - LLM Model: {config.llm.model_name}")
print(f"  - Vector Store: {config.vector_store.provider}")
print(f"  - Text Embedding Model: {config.embedding.text_model}")
print(f"  - Supported Formats: {config.document_processing.supported_formats}")

## 2. Initialize the RAG System

In [ ]:
# Initialize the multimodal RAG system
rag_system = MultimodalRAG(config)
print("🚀 Multimodal RAG system initialized!")

## 3. Document Ingestion

Let's ingest some sample documents. You can place your files in the `../data/raw/` directory.

In [ ]:
# Check what files are available for ingestion
raw_data_dir = Path("../data/raw")
if raw_data_dir.exists():
    files = list(raw_data_dir.glob("*"))
    print(f"📁 Found {len(files)} files in raw data directory:")
    for file in files[:10]:  # Show first 10 files
        print(f"  - {file.name} ({file.suffix})")
else:
    print("📁 Raw data directory not found. Creating sample structure...")
    raw_data_dir.mkdir(parents=True, exist_ok=True)
    print("   Place your documents in ../data/raw/ to continue")

### Ingest a Single File

Let's create a sample text file and ingest it:

In [ ]:
# Create a sample document
sample_doc = raw_data_dir / "sample_document.txt"
sample_content = """
# Multimodal RAG System Overview

The multimodal RAG system is designed to handle various types of content:

## Text Documents
- PDF files with complex layouts
- Microsoft Word documents
- Plain text and Markdown files
- Web pages and HTML content

## Visual Content
- Images with OCR text extraction
- Charts and diagrams
- Screenshots and photographs
- Scanned documents

## Audio Content
- Voice recordings and meetings
- Lectures and presentations
- Podcasts and interviews
- Music with lyrics

The system uses advanced embedding models to create semantic representations
of all content types, enabling unified search and retrieval across modalities.

Key features include:
- Offline operation with local LLMs
- High-quality embeddings for semantic search
- Configurable chunking and processing
- Scalable vector storage
- Context-aware response generation
"""

with open(sample_doc, 'w', encoding='utf-8') as f:
    f.write(sample_content)

print(f"📄 Created sample document: {sample_doc}")

In [ ]:
# Ingest the sample document
result = await rag_system.ingest_file(str(sample_doc))

print("✅ Document ingested successfully!")
print(f"📊 Ingestion results:")
print(f"  - File: {result['file_path']}")
print(f"  - File type: {result['file_type']}")
print(f"  - Chunks created: {result['content_chunks']}")
print(f"  - Status: {result['status']}")

### Batch Ingestion

If you have multiple files, you can ingest them all at once:

In [ ]:
# Ingest all files in the raw data directory
if list(raw_data_dir.glob("*")):
    results = await rag_system.ingest_directory(str(raw_data_dir))
    
    # Show summary
    successful = [r for r in results if r.get('status') == 'success']
    failed = [r for r in results if r.get('status') == 'error']
    total_chunks = sum(r.get('content_chunks', 0) for r in successful)
    
    print(f"📊 Batch ingestion complete:")
    print(f"  - Total files: {len(results)}")
    print(f"  - Successful: {len(successful)}")
    print(f"  - Failed: {len(failed)}")
    print(f"  - Total chunks: {total_chunks}")
    
    if failed:
        print("\n❌ Failed files:")
        for fail in failed:
            print(f"  - {fail.get('error', 'Unknown error')}")
else:
    print("📁 No files found for batch ingestion")

## 4. System Statistics

Let's check the current state of the system:

In [ ]:
# Get system statistics
stats = await rag_system.get_system_stats()

print("📊 System Statistics:")
for key, value in stats.items():
    print(f"  - {key}: {value}")

## 5. Querying the System

Now let's query the system with different types of questions:

### Basic Text Query

In [ ]:
# Simple question about content
query = "What types of content can the multimodal RAG system handle?"

result = await rag_system.query(
    query=query,
    query_type="text",
    top_k=3,
    include_sources=True
)

print(f"❓ Query: {query}")
print(f"\n🤖 Response:")
print(result['response'])
print(f"\n📚 Retrieved {result['retrieved_count']} relevant chunks")

### Show Source Documents

In [ ]:
# Display the sources used in the response
if result.get('sources'):
    print("\n📖 Source Documents:")
    for i, source in enumerate(result['sources'], 1):
        doc = source.get('document', {})
        score = source.get('score', 0)
        metadata = doc.get('metadata', {})
        
        print(f"\n{i}. **Similarity Score**: {score:.3f}")
        print(f"   **Source**: {metadata.get('source_file', 'Unknown')}")
        print(f"   **Content**: {doc.get('text', '')[:200]}...")

### More Complex Queries

In [ ]:
# Ask about specific features
queries = [
    "What are the key features of the system?",
    "How does the system handle audio content?",
    "What embedding models are used?",
    "What are the advantages of offline operation?"
]

for query in queries:
    result = await rag_system.query(query, top_k=2)
    
    print(f"\n{'='*50}")
    print(f"❓ {query}")
    print(f"\n🤖 {result['response']}")
    print(f"📊 Confidence: {result.get('confidence_scores', [])[:1]}")

## 6. Advanced Features

### Save and Load Index

In [ ]:
# Save the current index
await rag_system.save_index()
print("💾 Index saved successfully!")

# The index is automatically loaded on system initialization,
# but you can also load it explicitly:
# await rag_system.load_index()
# print("📥 Index loaded successfully!")

### Interactive Query Loop

In [ ]:
# Interactive query session
print("🔄 Starting interactive query session...")
print("Type 'exit' to stop, or ask any question about your documents")

# Note: This might not work well in some Jupyter environments
# For better interactive experience, use the CLI script: scripts/query.py -i

try:
    while True:
        user_query = input("\n❓ Your question: ").strip()
        
        if user_query.lower() in ['exit', 'quit', 'stop']:
            print("👋 Goodbye!")
            break
        
        if not user_query:
            continue
        
        # Process the query
        result = await rag_system.query(user_query, top_k=3)
        print(f"\n🤖 {result['response']}")
        
except KeyboardInterrupt:
    print("\n👋 Session ended by user")
except Exception as e:
    print(f"❌ Error in interactive session: {e}")
    print("💡 Tip: Use the CLI script for better interactive experience")

## 7. Cleanup and Next Steps

In [ ]:
# Final system statistics
final_stats = await rag_system.get_system_stats()

print("🎯 Demo Complete!")
print("\n📊 Final System State:")
for key, value in final_stats.items():
    print(f"  - {key}: {value}")

print("\n🚀 Next Steps:")
print("  1. Add your own documents to ../data/raw/")
print("  2. Try the CLI scripts: scripts/ingest.py and scripts/query.py")
print("  3. Experiment with different query types and parameters")
print("  4. Configure different embedding models or LLMs")
print("  5. Explore the Docker deployment option")

## Optional: Clear Index

Uncomment and run this cell if you want to clear the index for a fresh start:

In [ ]:
# Uncomment to clear the index
# await rag_system.clear_index()
# print("🗑️ Index cleared successfully!")